# DISA Database design

## 1. Migrating from MongoDB

The first task in recreating the DISA database was migrating data from the existing Mongo db. First, let's take a look at a sample record from the Mongo db (**disamg**), the contents of which have been dumped into `latest_entries.json`. 

In [22]:
import datetime as dt
import pandas as pd

from collections import namedtuple, Counter, defaultdict

import json
import random
import pprint

with open('data/latest-entries.json') as f:
    data = json.load(f)

In [23]:
data[0]

{'_id': {'$oid': '598b4c8baecb6c0001a29e25'},
 'person': {'names': [{'firstName': 'Elizabeth',
    'lastName': '',
    'type': 'English'}],
  'tribe': 'Unspecified',
  'origin': '',
  'race': 'Indian',
  'sex': 'Female',
  'typeKindOfEnslavement': 'Servant',
  'vocation': '',
  'father': {'name': {'firstName': '', 'lastName': ''},
   'race': '',
   'origin': '',
   'status': '',
   'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
    'vocation': ''}},
  'mother': {'name': {'firstName': '', 'lastName': ''},
   'race': '',
   'origin': '',
   'status': '',
   'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
    'vocation': ''}},
  'children': []},
 'owner': {'name': {'firstName': '', 'lastName': 'Duglas', 'title': 'Mr. '},
  'vocation': ''},
 'dateOfRunaway': {'month': ''},
 'dateOfEmancipation': {'month': ''},
 'buyer': {'firstName': '', 'lastName': ''},
 'seller': {'firstName': '', 'lastName': ''},
 'dateOfSale': {'month': ''},
 'dateOfMarriage': {'mon

We see a number of potential issues here. Let's consider each in turn.

### MongoDB Document Structure

Each document in any Mongo db is a JSON object, which itself can contain an arbitrary number of JSON objects. Here are the top-level attributes for all documents in **disamg**:

In [24]:
all_attrs = { k for d in data for k in d.keys() }
for a in sorted(list(all_attrs)):
    print(a)

_id
additionalInformation
buyer
dateOfDeath
dateOfEmancipation
dateOfMarriage
dateOfRunaway
dateOfSale
document
meta
owner
person
researcherNotes
seller


This is an odd assortment of labels; it's not immediately clear what each document in **disamg** represents, and what its relationship is to the data it contains. Some guesses:
* `document` identifies the historical resource under review by the human user, with `_id`, `meta`, `additionalInformation`, and `researcherNotes` all containing some administrative-level information.
* Considering the context of the DISA project, `person` is the central data point, with `buyer`, `seller`, `owner`, and the 5 `dateOf*` fields referring back to `person` (ie, 'owner of `person`, 'date of death of `person`, etc.).

If this reading is correct, there are some problems with the structure. Looking at the contents of the `person` object, we see fields for `mother`, `father`, `origin`, etc. We expect that these also refer back to the containing `person` object; i.e., `mother` of `person`. And looking into the `mother` object, we see a contained `owner` object (the same with `father`). This is presumably the `owner` of the `mother`. But why is `owner` a child of `mother` here, where `owner` is a sibling of `person` at the top level?

Because Mongo dbs are schemaless, there's no guarantees of consistency either between documents or within an individual document. While this flexibility is useful in getting a db up and running quickly, it presents challenges when it comes time to actually query the system and extract some useful information, as we attempt to do now. If we're to gain a better sense of the contents and purposes of this db, we'll need to impose some more structure on it.

The attributes `researcherNotes` and `additionalInformation` are both text strings; all the rest of the attributes are themselves JSON objects containing more data. We'll take a poll of these object-attributes, and see what fields are available in them. Once we can predict the data structure a little better, we'll see how much of this data is empty. The sample document shows several null values. Learning more about the presence and absence of data may provide us with some more clues.

In [25]:
str_vals = { 'additionalInformation', 'researcherNotes' }
obj_vals = { 'buyer', 'dateOfDeath', 'dateOfEmancipation', 'dateOfMarriage', 'dateOfRunaway',
            'dateOfSale', 'document', 'meta', 'owner', 'person', 'seller', '_id' }

from collections import Counter
count_attrs = { a: Counter() for a in all_attrs if a not in str_vals }
count_attrs['researcherNotes'] = 0
count_attrs['additionalInformation'] = 0
total_docs = 0
for d in data:
    total_docs += 1
    for attr in all_attrs:
        v = d.get(attr)
        if attr in str_vals and v is not None:
            count_attrs[attr] += 1
        elif v:
            for n in v.keys():
                count_attrs[attr][n] += 1

print('Total Mongo Documents: {}\n'.format(total_docs))
print('Document attribute counts\n')
for a in sorted(list(count_attrs.keys())):
    print(a)
    if a in obj_vals:
        for v, c in count_attrs[a].items():
            print('\t{}:{}'.format(v, c))
    else:
        print('\t',count_attrs[a])
    print()

Total Mongo Documents: 493

Document attribute counts

_id
	$oid:493

additionalInformation
	 493

buyer
	firstName:493
	lastName:493

dateOfDeath
	month:353
	year:1
	day:1

dateOfEmancipation
	month:493
	year:31
	day:32

dateOfMarriage
	month:493
	year:2
	day:2

dateOfRunaway
	month:493
	year:68
	day:58

dateOfSale
	month:493
	year:1
	day:1

document
	date:493
	stringLocation:493
	colonyState:493
	nationalContext:493
	sourceType:493
	recordType:493
	citation:493
	locale:3

meta
	stage:493
	creator:493
	lastModified:493
	updatedBy:493
	idPrefix:493
	idSuffix:493

owner
	name:493
	vocation:493

person
	names:493
	tribe:493
	origin:493
	race:493
	sex:493
	typeKindOfEnslavement:493
	vocation:493
	father:493
	mother:493
	children:493
	age:405

researcherNotes
	 493

seller
	firstName:493
	lastName:493



The attribute counts look reasonably consistent with the total record count. The biggest discrepencies are in the `dateOf*` objects. Notably, we see that there are only 353 `dateOfDeath` attributes, likely indicating that this was added later than the other attributes. Looking back at the sample record, we see that an empty date is stored as `{'month': ''}`; and that probably accounts for why there is a difference between `month`, `year`, and `day` counts for individual `dateOf` attributes. We'll guess at empty data structures for other attribute values, and see just how much this data is empty. 

We'll gloss over the `person` object for now. We see that, with the exception of `age`, all of the other attributes are reliably present. We also know from looking at the sample record that the `person` data is deeply nested, so this will require some special treatment. Instead, we'll focus on the other data here, and loop back to `person` later.

In [26]:
empty_attr = {
    '_id' : [{'$oid': ''}],
    'additionalInformation': [''],
    'buyer': [{'firstName': '', 'lastName': ''}],
    'dateOfRunaway': [{'month': ''}],
    'dateOfSale': [{'month': ''}],
    'dateOfMarriage': [{'month': ''}],
    'dateOfEmancipation': [{'month': ''}],
    'dateOfDeath': [{'month': ''}],
    'document': [{'date': {'month': ''}, 'stringLocation': '', 'colonyState': '', 'nationalContext': '',
                'sourceType': '', 'recordType': '', 'citation': ''},
                {'date': {'month': ''}, 'stringLocation': '', 'colonyState': '', 'nationalContext': '',
                'sourceType': '', 'recordType': '', 'citation': '', 'locale': ''}],
    'meta': [{'stage': '', 'creator': '', 'lastModified': '', 'updatedBy': '', 'idPrefix': '', 'idSuffix': ''}],
    'owner': [{'name': {'firstName': '', 'lastName': '', 'title': ''}, 'vocation': ''}],
    'researcherNotes': [''],
    'seller': [{'firstName': '', 'lastName': ''}]
}

count_empty = { k: 0 for k in empty_attr.keys() }
for d in data:
    for e, checks in empty_attr.items():
        v = d.get(e)
        if not v:
            count_empty[e] += 1
            continue
        for c in checks:
            if v == c:
                count_empty[e] += 1

print('Empty attribute counts\n')
for a in sorted(list(count_empty.keys())):
    print('{0}: {1} empty ({2:.2f}% non-empty)\n'.format(a, count_empty[a], 100-(count_empty[a]/total_docs)*100))

Empty attribute counts

_id: 0 empty (100.00% non-empty)

additionalInformation: 109 empty (77.89% non-empty)

buyer: 493 empty (0.00% non-empty)

dateOfDeath: 492 empty (0.20% non-empty)

dateOfEmancipation: 461 empty (6.49% non-empty)

dateOfMarriage: 491 empty (0.41% non-empty)

dateOfRunaway: 425 empty (13.79% non-empty)

dateOfSale: 492 empty (0.20% non-empty)

document: 3 empty (99.39% non-empty)

meta: 0 empty (100.00% non-empty)

owner: 75 empty (84.79% non-empty)

researcherNotes: 381 empty (22.72% non-empty)

seller: 486 empty (1.42% non-empty)



Some quick analysis:
* `buyer`, `seller`, `dateOfSale`, `dateOfMarriage`, `dateOfDeath` are almost entirely empty
* `owner` and `additionalInformation` are about 80% present
* `researcherNotes`, `dateOfEmancipation` and `dateOfRunaway` are less common, but still occasionally filled out
* `_id` and `meta` are present on all **disamg** documents

What about those 3 empty `document` values? Let's examine:

In [27]:
for d in data:
    if d['document'] in empty_attr['document']:
        for k,v in d.items():
            print('{}:\n\t{}'.format(k,v))
        print('='*100)

_id:
	{'$oid': '59b98bebdf4c6f000198331e'}
person:
	{'names': [{'firstName': 'FIRST NAME', 'lastName': '', 'type': 'English'}, {'firstName': '', 'lastName': 'LAST NAME', 'type': 'Indian'}], 'tribe': '', 'origin': '', 'race': '', 'sex': '', 'typeKindOfEnslavement': '', 'vocation': '', 'father': {'name': {'firstName': '', 'lastName': ''}, 'race': '', 'origin': '', 'status': '', 'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''}, 'vocation': ''}}, 'mother': {'name': {'firstName': '', 'lastName': ''}, 'race': '', 'origin': '', 'status': '', 'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''}, 'vocation': ''}}, 'children': []}
owner:
	{'name': {'firstName': '', 'lastName': '', 'title': ''}, 'vocation': ''}
dateOfRunaway:
	{'month': ''}
dateOfEmancipation:
	{'month': ''}
buyer:
	{'firstName': '', 'lastName': ''}
seller:
	{'firstName': '', 'lastName': ''}
dateOfSale:
	{'month': ''}
dateOfMarriage:
	{'month': ''}
document:
	{'date': {'month': ''}, 'stringLocation': '

Two of these appear to be test documents, meaning there is only 1 of the 493 without `document` data. We'll count this as an error.

If we were desiging an entity-relationship diagram for **disamg** as-is, it might look something like this:

![title](img/DISA_er_1.png)

We'll consider all nested object-attributes as independent entities for now, and the `additionalInformation` and `researcherNotes` fields as as entity attributes attendant on the Mongo document (whatever that might represent). A **disamg** document has a mandatory relationship with `_id`, `meta`, `document`, and `person`, since these are present on nearly every **disamg** document. This may be a many-to-one relationship: many **disamg** documents might be associated with the same historical `document`, for example. For now, we can be sure that each **disamg** document is associated with 1 and only 1 of these entities, since none of the attributes are multivalued.

Relationships between a document and the remaining entities -- `buyer`, `seller`, `owner`, and `dateOf*` -- are optional, given by their inconsistent presence in our analysis. These too are unary relationships, since again none of them are multivalued. However, here that seems potentially problematic. If our hunch is correct -- that each **disamg** document stands in for an enslaved person -- we might expect `owner` or `dateOfSale` to be multivalued, since someone might be sold multiple times over their lifetime. Moreover, what's the relationship between `buyer`, `seller`, and `owner`? One might expect that the `buyer` and `seller` of a person would be an `owner` as well.

### Defining DISA-Mongo Entities 

Before diving into potential improvements, let's continue our examination of the existing **disamg** entities.  First, we'll look at `_id` and `meta`. These likely contain administrative information -- especially, fields that look like IDs -- and may be helpful in determining just what a **disamg** document represents.

In [28]:
unique_data = {
    'id_oid': set(),
    'meta_id_prefix': set(),
    'meta_id_suffix': set(),
    'meta_id_stage': set(),
    'meta_creator': set(),
    'meta_updated_by': set(),
    'meta_modified': set(),
}

for d in data:
    unique_data['id_oid'].add(d['_id']['$oid'])
    unique_data['meta_id_prefix'].add(d['meta']['idPrefix'])
    unique_data['meta_id_suffix'].add(d['meta']['idSuffix'])
    unique_data['meta_id_stage'].add(d['meta']['stage'])
    unique_data['meta_creator'].add(d['meta']['creator'])
    unique_data['meta_updated_by'].add(d['meta']['updatedBy'])
    unique_data['meta_modified'].add(d['meta']['lastModified'])

for att, d in unique_data.items():
    d_list = list(d)
    print("Number of unique {{{}}} values: {}".format(att, len(d_list)))
    print("\tsamples: {}\n".format(d_list[:8]))

Number of unique {id_oid} values: 493
	samples: ['5a6fe1201b47910001a53ab2', '5a6fe8901b47910001a53ab9', '5a6a0b6c1b47910001a53a9d', '5a383cc8df4c6f000198336d', '5b048fb8de60e081fbda37fd', '5ad6b2692a3c60f43767427c', '5ad036b12a3c601621674272', '5a6a0a151b47910001a53a9a']

Number of unique {meta_id_prefix} values: 1
	samples: [10]

Number of unique {meta_id_suffix} values: 493
	samples: [10240, 10241, 10242, 10243, 10244, 10245, 10246, 10247]

Number of unique {meta_id_stage} values: 2
	samples: ['Internal', 'Draft']

Number of unique {meta_creator} values: 6
	samples: ['106123372953260397156', '112148132795694739523', '112487255676465508755', '113112063790792171857', '109503335312366098524', '106895219236186746888']

Number of unique {meta_updated_by} values: 8
	samples: ['106123372953260397156', '112487255676465508755', '112148132795694739523', '103795391716629952261', '113112063790792171857', '109503335312366098524', '117289295548725522159', '106895219236186746888']

Number of uniqu

### Notes
* `oid` looks like a nonsense string, probably platform-dependent. Uninterpretable, and probably not particularly useful anyway 
* `lastModified`, `oid`, and `idSuffix` values are unique for each of the 493 **disamg** documents
* there is only 1 vaule for `idPrefix`; presumably this is intended to be joined with ID suffix, although its purpose is unclear
* `lastModified` probably tracking when each document was last edited
* `creator` and `updatedBy` probably to point to user IDs, tracking who has edited each document. Again this is uninterpretable, although the Mongo db has another collection with user information
* `stage` maybe tracks the status of a document; meaning of `Draft` versus `Internal` is unknown

Moving forward, what's most useful here are the ID attributes, especially `idPrefix`+`idSuffix`. Clearly, each **disamg** document is intended as a uniquely addressable entity. We can also use some background knowledge here. We know that each person in DISA is intended to have some sort of public ID. The guess is that `idPrefix` and `idSuffix` serve this purpose. If so, this would confirm our suspicion that each **disamg** document stands in for an enslaved person. This returns us to some the questions we identified previously: why have a separate person attribute? And why model `_id`, `meta`, `owner`, `buyer`, and `seller` as sibling attributes to `person`?

If `idPrefix`+`idSuffix` identifies an enslaved person, a more reasonable model would have this datapoint contained within the `person` object, showing the attribute dependence on that entity (ie, `id` of `person`). Alternatively, if each **disamg** document corresponds to a person, why have the `person` attribute at all? Why not surface the data nested in `person`? That way, `idPrefix`+`idSuffix` would be sibling to `father`, `name`, `race`, etc; along with other top-level attributes like `owner`, `dateOfDeath`, etc.

In any event, reading `idPrefix`+`idSuffix` as an identifier for an enslaved person clarifies the significance and interpretation of each **disamg** document. We can reimagine our entity-relationship diagram like so:
![title](img/DISA_er_2.png)

This seems much more meaningful. Entities like `dateOfRunaway`, `owner`, and especially `_id` are now attendant on `Person`. However, this opens up new questions: does the information in `researcherNotes` and `additionalInformation` refer only to that `Person`? What about `document`; is that a `document` of a `Person`, the same as `owner` of a `Person`? Moreover, should data like `dateOfDeath` be an independent entity? It would make more sense as an attribute, since it's unlikely to be referenced by any other `Person`. Oppositely, we expect `owner` shouldn't be dependent on a single `Person`, since several `Person`s might be enslaved by a single `owner`. We know from our analysis that `owner` is one of the few data points that is consistently present, so this seems like an essential piece of data to capture.

Stepping back a bit, the Mongo document structure appears to be an almalgam of 2 designs. If our hunch is correct, each document stands in for an enslaved person. But, the other attributes -- especially `document` -- seem to point in another direction. This contains citation information, and also information related to the content of that citation: `colonyState`, `nationalContext`, `date`, etc.

Before diving into any of those issues, there's the more basic matter of `Person` being related to a `person` entity. This sounds redundant. As noted above, `person` probably contains data that pertains to `Person`. Now is a good time to dive into `person`, and get a better sense of what's there.

### Examining 'person' data

First, a quick review of the `person` object structure, plus a random example:

In [29]:
person_field_types = defaultdict(set)

for d in data:
    pers = d['person']
    for k,v in pers.items():
        person_field_types[k].add(type(v))

print("Person fields")
pprint.pprint(dict(person_field_types))
# Several iterations have shown #152 to be an especially useful sample
# rint = random.randrange(len(data))
# sample = data[rint]
print("\nSample #152")
pprint.pprint(data[152]['person'])

Person fields
{'age': {<class 'str'>},
 'children': {<class 'list'>},
 'father': {<class 'dict'>},
 'mother': {<class 'dict'>},
 'names': {<class 'list'>},
 'origin': {<class 'str'>},
 'race': {<class 'str'>},
 'sex': {<class 'str'>},
 'tribe': {<class 'str'>},
 'typeKindOfEnslavement': {<class 'str'>},
 'vocation': {<class 'str'>}}

Sample #152
{'age': '18',
 'children': [],
 'father': {'name': {'firstName': '', 'lastName': ''},
            'origin': '',
            'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
                      'vocation': ''},
            'race': '',
            'status': ''},
 'mother': {'name': {'firstName': '', 'lastName': ''},
            'origin': '',
            'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
                      'vocation': ''},
            'race': '',
            'status': ''},
 'names': [],
 'origin': '',
 'race': 'Indian',
 'sex': 'Male',
 'tribe': '',
 'typeKindOfEnslavement': 'Slave',
 'vocation

Two important takeaways here:
* all fields have only 1 type:
  * `names`, `children`: list
  * `father`, `mother`: dictionary
  * `age`, `origin`, `race`, `sex`, `tribe`, `typeKindEnslavement`, `vocation`: string
* all fields are optional, as most of the sample record is empty (including `names`)

We recall from our earlier analysis that, with the exception of `age`, all `person` fields are reliably present. Using the sample record as a template, we'll do a count of empty fields

In [30]:
empty = {
    'age': '',
    'children': [],
    'father': {'name': {'firstName': '', 'lastName': ''},
        'origin': '',
        'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
            'vocation': ''},
        'race': '',
        'status': ''},
    'mother': {'name': {'firstName': '', 'lastName': ''},
        'origin': '',
        'owner': {'name': {'firstName': '', 'lastName': '', 'title': ''},
            'vocation': ''},
        'race': '',
        'status': ''},
     'names': [],
     'origin': '',
     'race': '',
     'sex': '',
     'tribe': '',
     'typeKindOfEnslavement': '',
     'vocation': ''
}

missing_fields = Counter()
empty_fields = Counter()
for d in data:
    pers = d['person']
    # Reusing the union list put together in the last cell
    for field in person_field_types:
        v = pers.get(field)
        if field not in pers:
            missing_fields[field]+=1
        if not v or v == empty[field]:
            empty_fields[field]+=1

print("Missing field counts")
pprint.pprint(dict(missing_fields))

print("\nEmpty field counts\n")
for a in sorted(list(empty_fields.keys())):
    print('\t{0}: {1} empty ({2:.2f}% non-empty)\n'.format(a, empty_fields[a], 100-(empty_fields[a]/total_docs)*100))

Missing field counts
{'age': 88}

Empty field counts

	age: 90 empty (81.74% non-empty)

	children: 487 empty (1.22% non-empty)

	father: 487 empty (1.22% non-empty)

	mother: 481 empty (2.43% non-empty)

	names: 107 empty (78.30% non-empty)

	origin: 459 empty (6.90% non-empty)

	race: 147 empty (70.18% non-empty)

	sex: 17 empty (96.55% non-empty)

	tribe: 198 empty (59.84% non-empty)

	typeKindOfEnslavement: 100 empty (79.72% non-empty)

	vocation: 482 empty (2.23% non-empty)



Takeaways
* `age`, `sex`, `names`, and `typeKindEnslavement` are present on most records
* `race` and `tribe` are also mostly present, with `origin` occasionally present
* `children`, `father`, `mother`, and `vocation` are almost entirely empty

Our prior analysis regarding missing attributes was reconfirmed: only `age` is ever missing. Far more frequent is empty data fields. This is not necessarily a concern, and probably points more to the nature of the materials DISA is using as its sources. We might expect things data like `age` and `sex` to be commonly available in documents describing enslaved people; less so, data like `origin` or `vocation`.

We'll dump all the string attributes into a Pandas dataframe, and do more analysis there.

In [31]:
PersonStrings = namedtuple('PersonStrings',
    ['age', 'sex', 'race', 'enslavement', 'tribe', 'origin', 'vocation'])

pers_data = []
for d in data:
    pers = d['person']
    row = PersonStrings(
        age=pers.get('age',''),
        sex=pers.get('sex',''),
        race=pers.get('race',''),
        enslavement=pers.get('typeKindOfEnslavement',''),
        tribe=pers.get('tribe',''),
        origin=pers.get('origin',''),
        vocation=pers.get('vocation','')
    )
    pers_data.append(row)

df_patts = pd.DataFrame(pers_data)
df_patts.head(10)

,age,sex,race,enslavement,tribe,origin,vocation
0,,Female,Indian,Servant,Unspecified,,
1,19,Male,Indian,Slave,Unspecified,,
2,14,Male,Surinam Indian,Manslave,Unspecified,,
3,20,Female,Indian,Servant,,,
4,17,Female,Indian,Slave,,,
5,28,Male,Spanish Indian,Manslave,,,
6,23,Male,Indian,Man servant,,,
7,20,Female,Indian,Woman servant,,,
8,,Male,Indian,,Unspecified,,
9,,Female,Indian,,Unspecified,,


The dataframe looks good, although we're already seeing some bad data. "Indian" appears to be a common value for `race`. For a database of indigenous slavery in the Americas, this would seem to be redundant, as the label "Indian" might be applied to any indigenous person. Similarly, a value of "Unspecified" for `tribe` carries no information value. From a design perspective, this would typically be excluded from most databases.

Let's dig deeper:

In [32]:
for c in ['age', 'sex', 'race', 'enslavement', 'tribe', 'origin', 'vocation']:
    print(c)
    print("-"*20)
    print(df_patts[c].value_counts(),"\n")

age
--------------------
Not Mentioned            103
                          90
20                        39
30                        20
25                        18
18                        17
16                        15
Boy                       13
Not Mentioned             11
24                        10
Woman                      9
Girl                       8
22                         8
40                         8
17                         7
23                         7
14                         7
15                         7
9                          6
12                         6
Man                        6
10                         5
13                         5
5                          4
35                         4
28                         4
21                         4
Adult                      3
Not mentioned              3
30s                        3
19                         3
26                         3
11                         3
50                

We see a number of data issues here. Previously, it looked like only `vocation` was mostly empty. Now we see that, while the other attributes may have data present, the data appears to be of low information value. The most common value for `race` is "Indian"; `typeKindOfEnslavement`, "Slave"; and `tribe`, "Unspecified". `vocation` and `origin`, while mostly NULL values, are at least more accurate in their presentation. Add to that, there appears to be the inclusion of inconclusive data: values like "(maybe)", "(probably)", or other values followed by a "(?)".

We also see values that will pose issues for any future queries against the database. Some values appear to contain multiple values: "Sawyer & Carpenter", "Part African and Part Indian". Additionally, there is a lot of overlap between some values: "Indentured Servant", "Indenture", "Indenture, court-ordered"; "Mulatto", "Indian Mulatto", "West India Mulatto", etc. There is even overlap between attributes, as the distinction between `tribe` and `origin` is not clear. "Blanco" and "Valiente" appear in both columns ("The Valiente Nation" in `origin`). Moreover, the scope of `origin` itself is not clear; whether it's a physical location, or something else: "Bermuda", "Jamaica", versus "Spanish".

`age` requires extra attention. Again, we see vacuous data ("Not Mentioned"); we also see mixed datatypes. Some values are year integers ("14", "27"), others are ranges ("18-20", "14 or 15"), and others strings ("Old", "young girl", "Adult"). If the intention is to query this field by age as a number ("Show me all enslaved individuals younger than 12 years old"), this data will not be sufficient.

We won't engage in data cleanup yet. Some of this may be better handled at the business-logic level; some may require better user training and direction. For now, we're only concerned with the database design. `age` and `sex` are consistently available, with `age` being highly variable, and `sex` being highly consistent (either "Male" or "Female", with NULLs). We'll model these as attributes attendant on the `Person` entity.

The other fields -- `race`, `origin`, `tribe`, `vocation`, and `typeKindOfEnslavement` -- are more complex. These seem likely to be more stable than `age`, with consistent values. They also seem like good candidates for organizing and querying the database; ie., "Show me all enslaved people identified as Naragansett", "How many enslaved people worked as carpenters?". This suggets that these attributes are themselves independent entities from `Person`. And looking at their contents, we'll consider their relationship to `Person` as optional and many-to-many, since the same `Person` might have multiple `Race`s, etc., or no `Race`; and the same `Race` might describe multiple `Person`s. Surfacing this data from within the `person` attribute, our entity-relationship diagram now looks like this:
![title](img/DISA_er_3.png)

In [33]:
FlatRow = namedtuple('FlatRow', [
    #metadata
    'mongo_id', 'disa_id', 'modified_date', 'creator', 'last_updated_by',
    #name information
    'first_name', 'last_name', 'name_count',
    #personal details
    'tribe', 'origin', 'race', 'sex', 'age', 'enslavement', 'vocation',
    #personal event dates
    'date_runaway', 'date_sale', 'date_emancipation', 'date_marriage', 'date_death',
    #owner information
    'owner_first', 'owner_last', 'owner_title', 'owner_vocation',
    'buyer_first', 'buyer_last', 'seller_first', 'seller_last',
    #father information
    'father_first', 'father_last', 'father_origin', 'father_race', 'father_status',
    #father owner information
    'father_owner_first', 'father_owner_last', 'father_owner_title','father_owner_vocation',
    #mother information
    'mother_first', 'mother_last', 'mother_origin', 'mother_race', 'mother_status',
    #mother owner information
    'mother_owner_first', 'mother_owner_last', 'mother_owner_title', 'mother_owner_vocation',
    #child information
    'child_first', 'child_last', 'child_count',
    #location information
    'nation', 'state', 'location', 'locale',
    #document information
    'document_date', 'source', 'record', 'citation',
    #text blobs
    'addl_info', 'notes' ])

In [69]:
def get_date(dateData):
    if not dateData or dateData == {'month': ''} or dateData == {'year': '', 'month': '', 'day':''}:
        dateData = { 'day':1, 'month':1, 'year':1 }
    try:
        day = int( dateData.get('day',1) or 1)
        month = int( dateData.get('month',1) or 1)
        year = int( dateData.get('year',1) or 1)
    except:
        day = 1
        month = 1
        year = 1
    return dt.datetime(day=day, month=month, year=year) 

In [80]:
def nest(jd, *args):
    nest = jd
    for arg in args:
        try:
            nest = nest[arg]
        except:
            return ''
    return nest

def flatten(jd):
    row = FlatRow(
        mongo_id = nest(jd, '_id', '$oid'),
        disa_id = '{}-{}'.format(nest(jd,'meta','idPrefix'),nest(jd,'meta','idSuffix')),
        modified_date = dt.datetime.strptime(
            nest(jd, 'meta','lastModified'), '%Y-%m-%dT%H:%M:%S.%fZ'),
        creator = nest(jd, 'meta', 'creator'),
        last_updated_by = nest(jd, 'meta', 'updatedBy'),
        first_name = nest(jd, 'person', 'names', 0, 'firstName'),
        last_name = nest(jd, 'person', 'names', 0, 'firstName'),
        name_count = len( nest(jd, 'person', 'names') ),
        tribe = nest(jd, 'person', 'tribe'),
        origin = nest(jd, 'person', 'origin'),
        race = nest(jd, 'person', 'race'),
        sex = nest(jd, 'person', 'sex'),
        age = nest(jd, 'person', 'age'),
        enslavement = nest(jd, 'person', 'typeKindOfEnslavement'),
        vocation = nest(jd, 'person', 'vocation'),
        date_runaway = get_date( nest(jd, 'dateOfRunaway') ),
        date_sale = get_date( nest(jd, 'dateOfSale') ),
        date_emancipation = get_date( nest(jd, 'dateOfEmancipation') ),
        date_marriage = get_date( nest(jd, 'dateOfMarriage') ),
        date_death = get_date( nest(jd, 'dateOfDeath') ),
        owner_first = nest(jd, 'owner', 'firstName'),
        owner_last = nest(jd, 'owner', 'lastName'),
        owner_title = nest(jd, 'owner', 'title'),
        owner_vocation = nest(jd, 'owner', 'vocation'),
        buyer_first = nest(jd, 'buyer', 'firstName'),
        buyer_last = nest(jd, 'buyer', 'lastName'),
        seller_first = nest(jd, 'seller', 'firstName'),
        seller_last = nest(jd, 'seller', 'firstName'),
        father_first = nest(jd, 'person', 'father', 'name', 'firstName'),
        father_last = nest(jd, 'person', 'father', 'name', 'lastName'),
        father_origin = nest(jd, 'person', 'father', 'origin'),
        father_race = nest(jd, 'person', 'father', 'race'),
        father_status = nest(jd, 'person', 'father', 'status'),
        father_owner_first = nest(jd, 'person', 'father', 'owner', 'name', 'firstName'),
        father_owner_last = nest(jd, 'person', 'father', 'owner', 'name', 'lastName'),
        father_owner_title = nest(jd, 'person', 'father', 'owner', 'title'),
        father_owner_vocation = nest(jd, 'person', 'father', 'owner', 'vocation'),
        mother_first = nest(jd, 'person', 'mother', 'name', 'firstName'),
        mother_last = nest(jd, 'person', 'mother', 'name', 'lastName'),
        mother_origin = nest(jd, 'person', 'mother', 'origin'),
        mother_race = nest(jd, 'person', 'mother', 'race'),
        mother_status = nest(jd, 'person', 'mother', 'status'),
        mother_owner_first = nest(jd, 'person', 'mother', 'owner', 'name', 'firstName'),
        mother_owner_last = nest(jd, 'person', 'mother', 'owner', 'name', 'lastName'),
        mother_owner_title = nest(jd, 'person', 'mother', 'owner', 'title'),
        mother_owner_vocation = nest(jd, 'person', 'mother', 'owner', 'vocation'),
        child_first = nest(jd, 'person', 'children', 0, 'name', 'firstName'),
        child_last = nest(jd, 'person', 'children', 0, 'name', 'lastName'),
        child_count = len( nest(jd, 'person', 'children') ) ,
        nation = nest(jd, 'document', 'nationalContext'),
        state = nest(jd, 'document', 'colonyState'),
        location = nest(jd, 'document', 'stringLocation'),
        locale = nest(jd, 'document', 'locale'),
        document_date = get_date( nest(jd, 'document', 'date') ),
        source = nest(jd, 'document', 'sourceType'),
        record = nest(jd, 'document', 'recordType'),
        citation = nest(jd, 'document', 'citation'),
        addl_info = nest(jd, 'additionalInformation'),
        notes = nest(jd, 'researcherNotes') )

    return row

In [81]:
flat = [ flatten(rec) for rec in data ]
flat[0]

FlatRow(mongo_id='598b4c8baecb6c0001a29e25', disa_id='10-10002', modified_date=datetime.datetime(2018, 5, 14, 18, 32, 21, 539000), creator='112148132795694739523', last_updated_by='112487255676465508755', first_name='Elizabeth', last_name='Elizabeth', name_count=1, tribe='Unspecified', origin='', race='Indian', sex='Female', age='', enslavement='Servant', vocation='', date_runaway=datetime.datetime(1, 1, 1, 0, 0), date_sale=datetime.datetime(1, 1, 1, 0, 0), date_emancipation=datetime.datetime(1, 1, 1, 0, 0), date_marriage=datetime.datetime(1, 1, 1, 0, 0), date_death=datetime.datetime(1, 1, 1, 0, 0), owner_first='', owner_last='', owner_title='', owner_vocation='', buyer_first='', buyer_last='', seller_first='', seller_last='', father_first='', father_last='', father_origin='', father_race='', father_status='', father_owner_first='', father_owner_last='', father_owner_title='', father_owner_vocation='', mother_first='', mother_last='', mother_origin='', mother_race='', mother_status='',

In [82]:
disa = pd.DataFrame(flat)

In [83]:
disa

,mongo_id,disa_id,modified_date,creator,last_updated_by,first_name,last_name,name_count,tribe,origin,...,nation,state,location,locale,document_date,source,record,citation,addl_info,notes
0,598b4c8baecb6c0001a29e25,10-10002,2018-05-14 18:32:21.539,112148132795694739523,112487255676465508755,Elizabeth,Elizabeth,1,Unspecified,,...,British,Connecticut,New London,,1671-05-07 00:00:00,Book,,"Blake, The Later History of the First Church o...",,
1,599c82feaecb6c0001a29e26,10-10003,2018-06-04 21:26:02.976,112148132795694739523,103795391716629952261,Harry,Harry,1,Unspecified,,...,British,Narragansett (Rhode Island),Kingstown,,1704-06-12 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00004. America's Historical New...,"Details of advertisement: ""Ran away from Capt....",Although his owner is the aforementioned Natha...
2,599c8586aecb6c0001a29e27,10-10004,2018-05-22 22:58:51.373,112148132795694739523,109503335312366098524,Prince,Prince,1,Unspecified,,...,British,Massachusetts,Boston,,1704-11-06 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00006; America's Historical New...,"Details of advertisement: ""Ran-away on Wednesd...",
3,599c8922aecb6c0001a29e28,10-10005,2018-06-04 21:39:13.923,112148132795694739523,103795391716629952261,Hannah,Hannah,1,,,...,British,Massachusetts,Barnstable,,1706-02-17 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00012; America's Historical New...,"Details of advertisement: ""Ran-away the last S...",
4,599c8b4faecb6c0001a29e29,10-10006,2018-06-04 21:35:30.437,112148132795694739523,103795391716629952261,Grace,Grace,1,,,...,British,New York,,,1706-01-13 00:00:00,Newspaper,Runaway advertisement,Zotero ID: DISA00010; America's Historical New...,"Details of advertisement: ""Ran-away from her M...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,5b0498c0de60e011b9da3802,10-10490,2018-05-23 04:57:46.281,109503335312366098524,109503335312366098524,Milly,Milly,1,,,...,,Connecticut,Stonington,,1716-08-20 00:00:00,Newspaper,Runaway Advertisement,Zotero ID: DISA00070; America's Historical New...,"Details of advertisement: ""Ran away from Mr. D...",
489,5b0499bbde60e00a7cda3803,10-10491,2018-05-23 04:58:07.227,109503335312366098524,109503335312366098524,,,0,,,...,,Massachusetts,Boston,,1716-08-27 00:00:00,Newspaper,Advertisement of sale,Zotero ID: DISA00071; America's Historical New...,"Details of sale: ""A Spanish Indian Man Servant...",
490,5b049b52de60e03fe7da3804,10-10492,2018-05-22 22:37:09.231,109503335312366098524,109503335312366098524,,,1,,,...,,,,,0001-01-01 00:00:00,,,,,
491,5b049c9fde60e02bdada3805,10-10493,2018-05-23 17:23:40.210,109503335312366098524,112148132795694739523,,,0,,,...,British,Massachusetts,Boston,Sudbury Street,1716-10-01 00:00:00,Newspaper,Advertisement of sale,Zotero ID: DISA00073; America's Historical New...,"Details of sale: ""An Indian woman of about Twe...",


In [84]:
disa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 59 columns):
mongo_id                 493 non-null object
disa_id                  493 non-null object
modified_date            493 non-null datetime64[ns]
creator                  493 non-null object
last_updated_by          493 non-null object
first_name               493 non-null object
last_name                493 non-null object
name_count               493 non-null int64
tribe                    493 non-null object
origin                   493 non-null object
race                     493 non-null object
sex                      493 non-null object
age                      493 non-null object
enslavement              493 non-null object
vocation                 493 non-null object
date_runaway             493 non-null object
date_sale                493 non-null object
date_emancipation        493 non-null object
date_marriage            493 non-null object
date_death               493 non-

In [86]:
disa[ disa.child_count != 0 ]

,mongo_id,disa_id,modified_date,creator,last_updated_by,first_name,last_name,name_count,tribe,origin,...,nation,state,location,locale,document_date,source,record,citation,addl_info,notes
11,599cab56aecb6c0001a29e30,10-10013,2017-12-29 19:27:54.141,113112063790792171857,112148132795694739523,Rachel,Rachel,1,Mohegan,,...,British,Connecticut,New London,,1720-12-04 00:00:00,Archive,,"Connecticut Church Records, Index, New London ...",,
22,599f04bdaecb6c0001a29e3b,10-10024,2017-12-29 19:36:48.169,113112063790792171857,112148132795694739523,John,John,1,Unspecified,,...,British,Jamaica,Port Royal,,1749-04-27 00:00:00,Archive,Manumission,"Jamaica Archives, Manumission of Slaves; “Manu...",,Same name between child and slave.
91,59ea38c1df4c6f000198334a,10-10093,2018-05-31 16:32:56.581,112148132795694739523,113112063790792171857,Joan,Joan,1,,Pueblo de Pauto,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",,
92,5a0b84b6df4c6f000198334b,10-10094,2018-05-31 16:33:08.834,112148132795694739523,113112063790792171857,Catalina,Catalina,1,,Pueblo de Casanare,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",Married to Joan Pauto,Husband's ID in database: DISA-10-10093
119,5a3152addf4c6f0001983366,10-10121,2018-06-15 15:06:14.275,112148132795694739523,113112063790792171857,Joan,Joan,1,,,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",,Son's ID in database: DISA-10-10123
120,5a316c27df4c6f0001983367,10-10122,2018-06-15 15:06:41.665,112148132795694739523,113112063790792171857,Joan,Joan,1,,Casanave,...,Spanish,Trinidad,San Joseph de Oruña,,1688-07-22 00:00:00,Archive,Manumission,"DISA00107; CARTAS DE GOBERNADORE, Archivo Gene...",Joan is noted as a widower (viudo),


In [87]:
disa.iloc[92].date_marriage

datetime.datetime(1, 1, 1, 0, 0)

In [90]:
disa[ disa.addl_info.str.contains("owner") ]

,mongo_id,disa_id,modified_date,creator,last_updated_by,first_name,last_name,name_count,tribe,origin,...,nation,state,location,locale,document_date,source,record,citation,addl_info,notes
55,59c26e8edf4c6f0001983326,10-10057,2018-06-14 01:13:54.377,113112063790792171857,103795391716629952261,Israel,Israel,1,Unspecified,"Allentown, PA",...,British,Pennsylvania,Philadelphia,,1791-10-25 00:00:00,Newspaper,Runaway advertisement,DISA00094; America's Historical Newspapers [Ma...,Details of advertisement: “New Books. Just rec...,The same advertisement also appears in the Mar...
126,5a383cc8df4c6f000198336d,10-10128,2018-06-15 15:18:44.544,113112063790792171857,113112063790792171857,Pickle,Pickle,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-02-24 00:00:00,Archival,Registry,DISA00123; “Return of the Registry of Indians ...,"These records are very sparse. Same owner as ""...",
127,5a383e8ddf4c6f000198336e,10-10129,2018-06-15 15:18:58.994,113112063790792171857,113112063790792171857,Johnny,Johnny,1,Unspecified,,...,British,British Honduras,Mosquito Shore,,1777-02-24 00:00:00,Archive,Registry,DISA00123; “Return of the Registry of Indians ...,Same owner as that of Pickle,
129,5a3841bfdf4c6f0001983370,10-10131,2018-06-15 15:19:56.478,113112063790792171857,113112063790792171857,Jacob,Jacob,1,Unspecified,,...,British,British Honduras,Mosquito Shore,,1777-02-24 00:00:00,Archive,Registry,DISA00123; “Return of the Registry of Indians ...,Same owner as Catto.,
130,5a384276df4c6f0001983371,10-10132,2018-06-15 15:20:09.808,113112063790792171857,113112063790792171857,Catto,Catto,1,Unspecified,,...,British,British Honduras,Mosquito Shore,,1777-02-24 00:00:00,Archive,Registry,DISA00123; “Return of the Registry of Indians ...,Same owner as Jacob,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,5a949c181b47910001a53b2d,10-10431,2018-06-27 00:54:23.456,113112063790792171857,112148132795694739523,Mystilla,Mystilla,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",
430,5a949d191b47910001a53b2e,10-10432,2018-06-27 00:54:44.375,113112063790792171857,112148132795694739523,Susannah,Susannah,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",
431,5a949dca1b47910001a53b2f,10-10433,2018-06-27 00:54:53.798,113112063790792171857,112148132795694739523,Quash,Quash,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",
432,5a949e171b47910001a53b30,10-10434,2018-06-27 00:55:49.830,113112063790792171857,112148132795694739523,Docinda,Docinda,1,Unspecified,,...,British,British Honduras,Mosquito Coast,,1777-08-20 00:00:00,Archive,Registry,"Zotero ID: DISA00123, “Return of the Registry ...","Same owner as Tanton, Bricksome, Euler, Will, ...",


In [91]:
disa.disa_id.values

array(['10-10002', '10-10003', '10-10004', '10-10005', '10-10006',
       '10-10007', '10-10008', '10-10009', '10-10010', '10-10011',
       '10-10012', '10-10013', '10-10014', '10-10015', '10-10016',
       '10-10017', '10-10018', '10-10019', '10-10020', '10-10021',
       '10-10022', '10-10023', '10-10024', '10-10025', '10-10026',
       '10-10027', '10-10028', '10-10029', '10-10030', '10-10031',
       '10-10032', '10-10033', '10-10034', '10-10035', '10-10036',
       '10-10037', '10-10038', '10-10039', '10-10040', '10-10041',
       '10-10042', '10-10043', '10-10044', '10-10045', '10-10046',
       '10-10047', '10-10048', '10-10049', '10-10050', '10-10051',
       '10-10052', '10-10053', '10-10054', '10-10055', '10-10056',
       '10-10057', '10-10058', '10-10059', '10-10060', '10-10061',
       '10-10062', '10-10063', '10-10064', '10-10065', '10-10066',
       '10-10067', '10-10068', '10-10069', '10-10070', '10-10071',
       '10-10072', '10-10073', '10-10074', '10-10075', '10-100

In [94]:
disa.nation.value_counts()

British          365
                  54
Spanish           28
British           22
American          12
USA                6
British Coast      2
Honduras           2
United States      1
French             1
Name: nation, dtype: int64